<h1 align="center"> Airline Delays </h1>
<h2 align="center"> W261 - Final Project </h2>
<h5 align="center"> by Adam Sohn, Chandra Shekar Bikkanur, Jayesh Parikh, Tucker Anderson</h5>

<h2 align="center"> Business Question:</h2>

Flights that arrive late to the destination due to weather delays, technical delays, security delays and etc. cause economical loss and inconvenience to the passengers. Predicting if a flight is going to be delayed at the destination will give the airlines a headsup to act and mitigate the losses. For this analysis, we are going to look into the airlines data along with weather data to predict the arrival delay for a given flight in minutes.

<h2 align="center">EDA & Discussion of Challenges:</h2>

In [5]:
# Import libraries
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import functools
import dateutil.parser
import datetime
from math import atan2, cos, sin, radians, degrees

from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType, TimestampType
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, concat, lit, udf
from pyspark.sql import DataFrameNaFunctions
sqlContext = SQLContext(sc)

from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator, CrossValidatorModel
from pyspark.ml.stat import ChiSquareTest

<h5>Load Data:</h5>

For this analysis, we are going to import airlines data, weather data and airport codes data across United States.

In [7]:
#Read in airlines, weather, stations, airport codes dataset
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/201*.parquet")
weather_parquet = spark.read.option("header", "true")\
                      .parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/new_weather_parquet_177/weather201*a.parquet")
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/data/DEMO8/gsod/stations.csv.gz")
airport_codes = spark.read.csv('/FileStore/tables/airport_codes.csv', header="true", inferSchema="true")
airport_codes = airport_codes.selectExpr("`IATA Code` as code", "Latitude as lat", "Longitude as lon")

In [8]:
airlines.printSchema() # Check the data structure of the dataset

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: date (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: boolean (nullable = true)
-- DIVERTED: boolean (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)

In [9]:
airlines.count() # Check for total number of records

Out[5]: 31746841

In [10]:
len(airlines.columns) # Check for total number of columns

Out[6]: 56

In [11]:
display(airlines.sample(0.0000001, False)) # sample a fraction of records for data insight

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2017,3,8,31,4,2017-08-31,B6,11042,1104203,30647,CLE,"Cleveland, OH",OH,39,Ohio,44,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,947,959,12.0,12.0,0.0,0,0900-0959,11.0,1010,1126,6.0,1125,1132,7.0,7.0,0.0,0,1100-1159,false,false,98.0,93.0,76.0,1.0,563.0,3,null,null,null,null,null
2018,4,11,21,3,2018-11-21,WN,10792,1079206,30792,BUF,"Buffalo, NY",NY,36,New York,22,13232,1323202,30977,MDW,"Chicago, IL",IL,17,Illinois,41,1435,1430,-5.0,0.0,0.0,-1,1400-1459,12.0,1442,1511,3.0,1520,1514,-6.0,0.0,0.0,-1,1500-1559,false,false,105.0,104.0,89.0,1.0,468.0,2,null,null,null,null,null


In [12]:
def nullDataFrame(df):
  '''
  Create a pandas dataframe consisting of columns and corresponding null values for the given datftame, 'df' 
  '''
  null_feature_list = []
  count = df.count()
  for column in df.columns:
    nulls = df.filter(df[column].isNull()).count()
    nulls_perct = np.round((nulls/count)*100, 2)
    null_feature_list.append([column, nulls, nulls_perct])
  nullCounts_df = pd.DataFrame(np.array(null_feature_list), columns=['Feature_Name', 'Null_Counts', 'Percentage_Null_Counts'])
  return nullCounts_df

airlines_raw_nullCounts_df = nullDataFrame(airlines)
airlines_raw_nullCounts_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,YEAR,0,0.0
1,QUARTER,0,0.0
2,MONTH,0,0.0
3,DAY_OF_MONTH,0,0.0
4,DAY_OF_WEEK,0,0.0
5,FL_DATE,0,0.0
6,OP_UNIQUE_CARRIER,0,0.0
7,ORIGIN_AIRPORT_ID,0,0.0
8,ORIGIN_AIRPORT_SEQ_ID,0,0.0
9,ORIGIN_CITY_MARKET_ID,0,0.0


In [13]:
display(airlines.describe())

summary,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
count,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31274521,31269545,31269545,31269545,31269545,31746841,31260424,31260429,31244917,31244917,31746841,31244919,31176201,31176201,31176201,31176201,31746841,31746677,31178799,31178799,31746841,31746841,31746841,5799114,5799114,5799114,5799114,5799114
mean,2017.1512498204152,2.5174877084620797,6.552106365480585,15.749554640727876,3.9346285509162944,null,12668.724409461716,1266875.803290192,31729.315288031336,null,null,null,26.35374732245013,null,54.91906164774001,12668.666651116562,1266870.0274082704,31729.2951808339,null,null,null,26.354102948384693,null,54.919218135750896,1330.0884999550035,1334.2122192375064,9.855285614165476,12.909587811399238,0.18207943223989986,0.036036885090588944,null,16.830789563186986,1356.9563268309594,1464.4766360877195,7.5604571777227,1488.9034405659447,1468.8957719173475,4.615475952313754,12.966188215170925,0.18601095752494026,-0.20968071125792395,null,143.2167191860742,138.22906985609035,113.8502422431345,1.0,823.2170183483768,3.7652922065537164,19.98459350859459,3.2259498606166392,15.44036813209742,0.08916793151505557,25.364284785572416
stddev,1.431653281021503,1.1053295681781927,3.3994302561415295,8.774238088354528,1.991763538747178,null,1526.7397787182156,152673.7066902925,1289.4588026200715,null,null,null,16.539517798596844,null,26.577828324534646,1526.7212131574859,152671.85014169713,1289.419206153186,null,null,null,16.539679261968384,null,26.57807966993098,489.86848319644014,503.2922887741845,43.5052029370407,42.44165318434854,0.38590998608194244,2.161932356946247,null,9.488981863443778,504.9367808166726,531.9873729297799,5.92997944817499,516.8048646426242,536.3586689058154,45.59418015238943,42.14088584758871,0.3891155176322491,2.2975645036344483,null,74.73117735923344,74.33716296557806,72.24024903973572,0.0,607.6826683052021,2.3923501887692864,59.30797970625764,26.812025382335815,34.73908233877254,2.9147981743398184,48.60358147038267
min,2015,1,1,1,1,9E,10135,1013503,30070,ABE,"Aberdeen, SD",AK,1,Alabama,1,10135,1013503,30070,ABE,"Aberdeen, SD",AK,1,Alabama,1,1,1,-234.0,0.0,0.0,-2,0001-0559,0.0,1,1,0.0,1,1,-238.0,0.0,0.0,-2,0001-0559,-99.0,14.0,4.0,1.0,21.0,1,0.0,0.0,0.0,0.0,0.0
max,2019,4,12,31,7,YX,16869,1686901,36133,YUM,"Yuma, AZ",WY,78,Wyoming,93,16869,1686901,36133,YUM,"Yuma, AZ",WY,78,Wyoming,93,2359,2400,2755.0,2755.0,1.0,12,2300-2359,227.0,2400,2400,414.0,2400,2400,2695.0,2695.0,1.0,12,2300-2359,948.0,1604.0,1557.0,1.0,5095.0,11,2695.0,2692.0,1848.0,1078.0,2454.0


In [14]:
sample_airlines_df = airlines.sample(False, 0.0001, 2020)
airlines_pandas_df = sample_airlines_df.toPandas()

/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)

In [15]:
numeric_features = [x[0] for x in airlines.dtypes if x[1] == 'int' or x[1] == 'double']
airlines_pandas_df[numeric_features].hist(figsize=(30,30), bins=50)
display(plt.show())

In [16]:
airlines_pandas_df.corr() # Check the correlation of the features with ARR_DELAY

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN_STATE_FIPS,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST_STATE_FIPS,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
YEAR,1.000000,0.027245,0.036315,0.026719,-0.025623,-0.010927,-0.010924,-0.009716,0.021722,-0.037390,0.007064,0.007067,0.028988,0.032613,-0.038472,-0.027221,-0.027685,0.028977,0.030955,0.012619,0.017931,0.072703,-0.023727,-0.012337,-0.002311,-0.004713,-0.010057,0.026492,0.032724,0.015501,0.011243,-0.010194,-0.038534,-0.017232,-0.016805,-0.025913,NaN,-0.033147,-0.028839,0.033212,0.015593,-0.011671,NaN,0.062875
QUARTER,0.027245,1.000000,0.969916,0.012970,0.013014,0.021420,0.021421,0.050313,0.003572,-0.003701,0.013686,0.013686,0.026063,-0.021461,0.008866,0.012162,0.021252,-0.003205,-0.002406,-0.001524,0.004648,-0.000128,0.021392,0.002051,0.004912,0.010268,-0.000052,-0.009723,-0.004859,-0.017009,-0.007650,-0.045082,0.015040,0.007771,0.001922,0.001694,NaN,0.011972,0.013230,-0.025513,-0.025687,0.092382,NaN,0.001798
MONTH,0.036315,0.969916,1.000000,0.007653,0.015631,0.022241,0.022241,0.057682,0.005509,-0.002387,0.019026,0.019027,0.025237,-0.018354,0.014429,0.011358,0.017888,-0.006035,-0.005224,-0.004860,-0.000630,-0.002250,0.019985,0.004916,0.005283,0.015823,0.002964,-0.011140,-0.005986,-0.020296,-0.010392,-0.039038,0.014376,0.007283,0.002319,0.002308,NaN,0.011409,0.013342,-0.018053,-0.024879,0.088749,NaN,-0.002919
DAY_OF_MONTH,0.026719,0.012970,0.007653,1.000000,0.024595,-0.000954,-0.000954,-0.025161,-0.019787,0.003617,-0.027405,-0.027405,-0.031011,0.005575,0.001240,-0.038648,-0.039347,-0.026265,-0.023744,-0.018424,-0.017345,0.003060,-0.033409,-0.015103,0.012239,-0.011522,-0.014857,-0.027449,-0.024578,-0.029835,-0.019329,-0.032208,0.020449,0.028223,0.023105,0.022431,NaN,0.025627,0.023623,-0.012056,-0.018533,0.046834,NaN,-0.031233
DAY_OF_WEEK,-0.025623,0.013014,0.015631,0.024595,1.000000,-0.011436,-0.011437,-0.004043,-0.005203,-0.014304,0.043460,0.043461,0.058038,-0.013205,-0.049296,0.002067,0.007483,-0.009442,-0.010738,-0.000761,-0.003261,-0.052986,0.009320,0.012883,0.019687,0.019355,0.018121,-0.025822,-0.021702,-0.005155,-0.019098,0.002249,-0.008434,0.039164,0.029569,0.035102,NaN,0.036655,0.036791,-0.057791,-0.028370,-0.088815,NaN,0.036059
ORIGIN_AIRPORT_ID,-0.010927,0.021420,0.022241,-0.000954,-0.011436,1.000000,1.000000,0.635169,-0.101700,0.251371,0.012615,0.012616,-0.003904,-0.029606,0.116429,-0.040424,-0.035854,-0.025605,-0.021149,-0.019009,-0.023874,-0.029176,-0.038927,0.012933,0.010375,0.002420,0.009250,-0.019422,-0.015105,0.002071,-0.017611,-0.017611,-0.003331,0.045404,0.048293,0.052135,NaN,0.070723,0.074988,-0.078693,-0.024214,0.052672,NaN,-0.004275
ORIGIN_AIRPORT_SEQ_ID,-0.010924,0.021421,0.022241,-0.000954,-0.011437,1.000000,1.000000,0.635168,-0.101701,0.251371,0.012616,0.012616,-0.003903,-0.029606,0.116429,-0.040424,-0.035854,-0.025605,-0.021149,-0.019009,-0.023873,-0.029176,-0.038926,0.012933,0.010374,0.002420,0.009250,-0.019422,-0.015105,0.002071,-0.017611,-0.017612,-0.003331,0.045403,0.048293,0.052135,NaN,0.070722,0.074988,-0.078692,-0.024213,0.052672,NaN,-0.004275
ORIGIN_CITY_MARKET_ID,-0.009716,0.050313,0.057682,-0.025161,-0.004043,0.635169,0.635168,1.000000,0.008020,0.065779,-0.004904,-0.004904,-0.041660,-0.028528,0.019490,-0.066390,-0.068907,-0.065964,-0.058110,-0.069272,-0.070384,-0.092384,-0.068669,-0.039690,0.070359,-0.049016,-0.035443,-0.059496,-0.052885,-0.052600,-0.065244,-0.014584,-0.022242,-0.003427,-0.000005,0.005802,NaN,0.008401,0.010437,-0.045065,0.006605,0.021613,NaN,-0.060296
ORIGIN_STATE_FIPS,0.021722,0.003572,0.005509,-0.019787,-0.005203,-0.1

In [17]:
weather_parquet.printSchema() # data structure of weather data

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- SOURCE: short (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- QUALITY_CONTROL: string (nullable = true)
-- WND: string (nullable = true)
-- CIG: string (nullable = true)
-- VIS: string (nullable = true)
-- TMP: string (nullable = true)
-- DEW: string (nullable = true)
-- SLP: string (nullable = true)
-- AW1: string (nullable = true)
-- GA1: string (nullable = true)
-- GA2: string (nullable = true)
-- GA3: string (nullable = true)
-- GA4: string (nullable = true)
-- GE1: string (nullable = true)
-- GF1: string (nullable = true)
-- KA1: string (nullable = true)
-- KA2: string (nullable = true)
-- MA1: string (nullable = true)
-- MD1: string (nullable = true)
-- MW1: string (nullable = true)
-- MW2: string (nullable = true)
-- OC1: string (nullable = true)
-- OD1: string (nullable = true)
-- OD2: string (nullable = true)
-- REM: string (nullable = true)
-- EQD: string (nullable = true)
-- AW2: string (nullable = true)
-- AX4: string (nullable = true)
-- GD1: string (nullable = true)
-- AW5: string (nullable = true)
-- GN1: string (nullable = true)
-- AJ1: string (nullable = true)
-- AW3: string (nullable = true)
-- MK1: string (nullable = true)
-- KA4: string (nullable = true)
-- GG3: string (nullable = true)
-- AN1: string (nullable = true)
-- RH1: string (nullable = true)
-- AU5: string (nullable = true)
-- HL1: string (nullable = true)
-- OB1: string (nullable = true)
-- AT8: string (nullable = true)
-- AW7: string (nullable = true)
-- AZ1: string (nullable = true)
-- CH1: string (nullable = true)
-- RH3: string (nullable = true)
-- GK1: string (nullable = true)
-- IB1: string (nullable = true)
-- AX1: string (nullable = true)
-- CT1: string (nullable = true)
-- AK1: string (nullable = true)
-- CN2: string (nullable = true)
-- OE1: string (nullable = true)
-- MW5: string (nullable = true)
-- AO1: string (nullable = true)
-- KA3: string (nullable = true)
-- AA3: string (nullable = true)
-- CR1: string (nullable = true)
-- CF2: string (nullable = true)
-- KB2: string (nullable = true)
-- GM1: string (nullable = true)
-- AT5: string (nullable = true)
-- AY2: string (nullable = true)
-- MW6: string (nullable = true)
-- MG1: string (nullable = true)
-- AH6: string (nullable = true)
-- AU2: string (nullable = true)
-- GD2: string (nullable = true)
-- AW4: string (nullable = true)
-- MF1: string (nullable = true)
-- AA1: string (nullable = true)
-- AH2: string (nullable = true)
-- AH3: string (nullable = true)
-- OE3: string (nullable = true)
-- AT6: string (nullable = true)
-- AL2: string (nullable = true)
-- AL3: string (nullable = true)
-- AX5: string (nullable = true)
-- IB2: string (nullable = true)
-- AI3: string (nullable = true)
-- CV3: string (nullable = true)
-- WA1: string (nullable = true)
-- GH1: string (nullable = true)
-- KF1: string (nullable = true)
-- CU2: string (nullable = true)
-- CT3: string (nullable = true)
-- SA1: string (nullable = true)
-- AU1: string (nullable = true)
-- KD2: string (nullable = true)
-- AI5: string (nullable = true)
-- GO1: string (nullable = true)
-- GD3: string (nullable = true)
-- CG3: string (nullable = true)
-- AI1: string (nullable = true)
-- AL1: string (nullable = true)
-- AW6: string (nullable = true)
-- MW4: string (nullable = true)
-- AX6: string (nullable = true)
-- CV1: string (nullable = true)
-- ME1: string (nullable = true)
-- KC2: string (nullable = true)
-- CN1: string (nullable = true)
-- UA1: string (nullable = true)
-- GD5: string (nullable = true)
-- UG2: string (nullable = true)
-- AT3: string (nullable = true)
-- AT4: string (nullable = true)
-- GJ1: string (nullable = true)
-- MV1: string (nullable = true)
-- GA5: string (nullable = true)
-- CT2: string (nullable = true)
-- CG2: string

In [18]:
weather_parquet.count() # to check the number of records in weather data

Out[14]: 626994336

In [19]:
len(weather_parquet.columns) # Check the number of columns in weather data

Out[15]: 177

In [20]:
display(weather_parquet.sample(0.000000001, False)) # sample a fraction of records for data insight

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
72515004725,2017-02-09T02:53:00.000+0000,7,42.2068,-75.98,486.2,"BINGHAMTON GREATER AP, NY US",FM-15,KBGM,V030,"360,5,N,0057,5","00457,5,M,N","016093,5,N,5","-0028,5","-0056,5","10082,5",,"07,5,+00457,5,99,9","08,5,+00884,5,99,9",,,"9,AGL ,+99999,+99999","99,99,9,99,9,99,9,00457,1,99,9,99,9",,,"10068,5,09485,5","8,9,008,9,+999,9",,,,,,MET10802/08/17 21:53:02 METAR KBGM 090253Z 36011KT 10SM BKN015 OVC029 M03/M06 A2973 RMK AO2 SLP082 T10281056 58008,,,,"3,99,1,+00457,5,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"4,99,1,+00884,5,9",,,"01,0000,9,5",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


<h2 align="center"> Feature Engineering:</h2>

<h5>Airlines & Weather Data Merge:</h5>

We will merge airlines and weather data sets to form a composite dataset.

In [23]:
def is_Weekend(x):
  """
  Function to determine if a given day of the week is a weekend_day(Friday, Saturday, Sunday)
  """
  if   x < 5: 
    return 0
  else: 
    return 1

def is_RushHour(x):
  """
  Function to determine if a given time of the day is rush hour (1600-2100)
  """
  if (x != None) and (x >= 1600) and (x <= 2100): 
    return 1
  else: 
    return 0
 
def preprocessAirlines(df):
  cols_to_keep = ['MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_DELAY', 'DEP_TIME_BLK', 'ARR_DELAY', 'ARR_TIME_BLK', 'CRS_ELAPSED_TIME', 'DISTANCE',  'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'IS_WEEKEND', 'DEP_RUSH_HOUR', 'ARR_RUSH_HOUR', 'DEP_TIME', 'CRS_DEP_TIME', 'ARR_TIME', 'CRS_ARR_TIME']
  cols_to_remove = [x for x in df.columns if x not in cols_to_keep]
  df = df.orderBy("FL_DATE") 
  df = df.filter(df.CANCELLED == False)
  df = df.filter(df.DIVERTED == False)
  df = df.withColumn('CARRIER_DELAY', f.when(df.CARRIER_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('WEATHER_DELAY', f.when(df.WEATHER_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('NAS_DELAY', f.when(df.NAS_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('SECURITY_DELAY', f.when(df.SECURITY_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('LATE_AIRCRAFT_DELAY', f.when(df.LATE_AIRCRAFT_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn("IS_WEEKEND", f.udf(is_Weekend, IntegerType())("DAY_OF_WEEK"))
  df = df.withColumn("DEP_RUSH_HOUR", f.udf(is_RushHour, IntegerType())("DEP_TIME"))
  df = df.withColumn("ARR_RUSH_HOUR", f.udf(is_RushHour, IntegerType())("CRS_ARR_TIME"))
  df = df.fillna(0, subset=['ARR_DELAY', 'DEP_DELAY'])  
  df = df.withColumn('ORIGIN_CARRIER', concat(col("ORIGIN"), lit("_"), col("OP_UNIQUE_CARRIER")))
  df = df.withColumn('DEST_CARRIER', concat(col("DEST"), lit("_"), col("OP_UNIQUE_CARRIER")))
  preprocessAirlines_df = df.drop(*cols_to_remove)
  return preprocessAirlines_df

In [24]:
def unionAll_fn(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

def US_fn(df):
    """
    Reduce df to US only to reduce size of dataset
    """
    # US is lat/long ranges according to format: [[(lat_low, lat_high),(long_low, long_high)], [(lat_low, lat_high),(long_low, long_high)]]
    US = [[(24,49),(-125,-67)],[(17,19),(-68,-65.5)], [(13,14),(144,145)], [(15,16),(145,146)], [(-15,-14), (-171,-170)], [(18,19),(-65.4,-64)], [(18,23),(-160,-154)], [(50,175),(-170,-103)]]  

    list_df = [] #empty list for parquet parts
    parquet_part = spark.range(0).drop("id") #empty spark df

    #Filtering for individual areas in US
    for item in US:
      parquet_part = df.filter((f.col('Latitude') > item[0][0]) & (f.col('Latitude') < item[0][1]) & (f.col('Longitude') > item[1][0]) & (f.col('Longitude') < item[1][1]))
      list_df.append(parquet_part)
    
    #Appending each individual US area
    parquet_us = unionAll_fn(list_df)

    return parquet_us

def reduce_split_cols_fn(weather_parquet_us):
    """
    Reduce weather dataset to columns of interest and return split columns with comma-separated values into multiple columns for each comma-separated value.
    """
    #Reduce weather dataset to columns of interest (high level) and return split columns with comma-separated values into multiple columns for each comma-separated value.
    weather_pre_split = weather_parquet_us.select('STATION','DATE','SOURCE','LATITUDE','LONGITUDE',f.split('WND', ',').alias('WND'),f.split('VIS', ',').alias('VIS'),f.split('SLP', ',').alias('SLP'),f.split('AA1', ',').alias('AA1'))
    df_sizes_WND = weather_pre_split.select(f.size('WND').alias('WND'))
    df_sizes_VIS = weather_pre_split.select(f.size('VIS').alias('VIS'))
    df_sizes_SLP = weather_pre_split.select(f.size('SLP').alias('SLP'))
    df_sizes_AA1 = weather_pre_split.select(f.size('AA1').alias('AA1'))
    df_max_WND = df_sizes_WND.agg(f.max('WND'))
    df_max_VIS = df_sizes_VIS.agg(f.max('VIS'))
    df_max_SLP = df_sizes_SLP.agg(f.max('SLP'))
    df_max_AA1 = df_sizes_AA1.agg(f.max('AA1'))
    nb_columns_WND = df_max_WND.collect()[0][0]
    nb_columns_VIS = df_max_VIS.collect()[0][0]
    nb_columns_SLP = df_max_SLP.collect()[0][0]
    nb_columns_AA1 = df_max_AA1.collect()[0][0]
    weather_post_split = weather_pre_split.select('STATION','DATE','SOURCE','LATITUDE','LONGITUDE',*[weather_pre_split['WND'][i] for i in range(nb_columns_WND)],*[weather_pre_split['VIS'][i] for i in range(nb_columns_VIS)],*[weather_pre_split['SLP'][i] for i in range(nb_columns_SLP)],*[weather_pre_split['AA1'][i] for i in range(nb_columns_AA1)])
  
    #Filtering out data with quality issues. All string values are indicative of quality issue
    fltr_msk = [
    f.col('WND[0]') != '999',
    f.col('WND[1]') != '2',
    f.col('WND[1]') != '3',
    f.col('WND[1]') != '6',
    f.col('WND[1]') != '7',
    f.col('WND[2]') != '9',
    f.col('WND[3]') != '9999',  
    f.col('WND[4]') != '2',
    f.col('WND[4]') != '3',
    f.col('WND[4]') != '6',
    f.col('WND[4]') != '7',
    f.col('VIS[0]') != '999999',
    f.col('VIS[1]') != '2',
    f.col('VIS[1]') != '3',
    f.col('VIS[1]') != '6',
    f.col('VIS[1]') != '7',
    f.col('VIS[2]') != '9',
    f.col('VIS[3]') != '2',
    f.col('VIS[3]') != '3',
    f.col('VIS[3]') != '6',
    f.col('VIS[3]') != '7',
    f.col('SLP[0]') != '99999',
    f.col('SLP[1]') != '2',
    f.col('SLP[1]') != '3',
    f.col('SLP[1]') != '6',
    f.col('SLP[1]') != '7',
    f.col('SLP[1]') != '9',
    f.col('AA1[0]') != '99',
    f.col('AA1[1]') != '9999',
    f.col('AA1[2]') != '9',
    f.col('AA1[3]') != '2',
    f.col('AA1[3]') != '3',
    f.col('AA1[3]') != '6',
    f.col('AA1[3]') != '7'
    ]
    weather_fltr = weather_post_split
    for i in fltr_msk:
      weather_fltr = weather_fltr.filter(i)

    #Reduce weather dataset to columns of interest (low level)
    weather_fltr_drop = weather_fltr.select('STATION','DATE','SOURCE','LATITUDE','LONGITUDE','WND[0]', 'WND[3]','VIS[0]','SLP[0]','AA1[0]')
    weather_fltr_drop = weather_fltr_drop.withColumnRenamed("DATE", "TIMESTAMP")

    return weather_fltr_drop

def distinct_station_fn(weather_fltr_drop):
    """
    For df input, return distinct stations for calculating closest stations to airports
    """
    weather_fltr_drop_distinct = weather_fltr_drop.select("STATION", "LATITUDE", "LONGITUDE").distinct()
    return weather_fltr_drop_distinct

def haversine_join_station_aircode_fn(airport_codes_df, weather_df):
    """
    For df input, return haversine distance
    """
    airport_codes_df.createOrReplaceTempView('airport_codes_us')
    weather_df.createOrReplaceTempView('stations_all')
    distance_query = "(SELECT airport_codes_us.code, stations_all.STATION, airport_codes_us.lat AS airport_lat, airport_codes_us.lon AS airport_lon, ( 3959 * acos(cos(radians(airport_codes_us.lat) ) * cos( radians( stations_all.LATITUDE ) ) * cos( radians( stations_all.LONGITUDE ) - radians(airport_codes_us.lon) ) + sin(radians(airport_codes_us.lat) ) * sin( radians( stations_all.LATITUDE ) ) ) ) AS airport_station_distance FROM airport_codes_us CROSS JOIN stations_all)"
    airports_stations_distance_all = spark.sql(distance_query)
    return airports_stations_distance_all
  
def airports_closest_stations_fn(airports_stations_distance_all):
    """
    For df input, return df with closest weather stations to airports
    """

    airports_stations_distance_all.createOrReplaceTempView('airports_stations_distance')
    closest_query = "(SELECT code AS airport_code, STATION AS station_name, airport_lat, airport_lon, airport_station_distance FROM airports_stations_distance ORDER BY airport_station_distance)"
    airports_closest_stations = spark.sql(closest_query)
  
    min_distance_query = "(SELECT code AS airport_code, STATION AS station_code, airport_lat, airport_lon, airport_station_distance FROM (SELECT *, row_number() over (partition by code order by airport_station_distance ASC) as seqnum from airports_stations_distance) airports_stations_distance where seqnum = 1)"
    airports_closest_station = spark.sql(min_distance_query)    

    MAX_ALLOWABLE_WEATHER_DISTANCE = 50.0
    airports_closest_station_filtered = airports_closest_station.filter(airports_closest_station.airport_station_distance < MAX_ALLOWABLE_WEATHER_DISTANCE)
    return airports_closest_station_filtered
  
def bearingClass_fn(flight_bearing, denominations=8):
    denom = 360/denominations
        
    if (int(flight_bearing) < 0 + denom/2) or (int(flight_bearing) > (7*denom) + (denom/2)):
      flight_bearing_class = "N"
    elif int(flight_bearing) <= denom + (denom/2):
      flight_bearing_class = "NW"
    elif int(flight_bearing) <= (2*denom) + (denom/2):
      flight_bearing_class = "W"
    elif int(flight_bearing) <= (3*denom) + (denom/2):
      flight_bearing_class = "SW"
    elif int(flight_bearing) <= (4*denom) + (denom/2):
      flight_bearing_class = "S"
    elif int(flight_bearing) <= (5*denom) + (denom/2):
      flight_bearing_class = "SE"
    elif int(flight_bearing) <= (6*denom) + (denom/2):
      flight_bearing_class = "E"
    elif int(flight_bearing) <= (7*denom) + (denom/2):
      flight_bearing_class = "NE"
    else:
      flight_bearing_class = "UNK"
      
    return flight_bearing_class
  
udfBearingClass_fn = udf(bearingClass_fn, StringType())

def bearingCalculation_fn(lat_a, lon_a, lat_b, lon_b):  
    lat_a_r, lat_b_r, lon_a_r, lon_b_r = radians(lat_a), radians(lat_b), radians(lon_a), radians(lon_b)
    delta_lon = lon_b - lon_a
    delta_lon_r = lon_b_r - lon_a_r
    X = cos(lat_b_r) * sin(delta_lon_r)
    Y = cos(lat_a_r) * sin(lat_b_r) - sin(lat_a_r) * cos(lat_b_r) * cos(delta_lon_r)
  
    flight_bearing = degrees(atan2(X, Y))
        
    flight_bearing_class = bearingClass_fn(flight_bearing)
  
    return flight_bearing_class
udfBearingCalculation_fn = udf(bearingCalculation_fn, StringType())

def join_closest_weather_airlines_fn(airlines_df, airports_closest_station_filtered):

    # add closest weather station to airlines dataset
    airlines_station_origin_filtered = airlines_df.join(airports_closest_station_filtered, airlines_df.ORIGIN == airports_closest_station_filtered.airport_code, how="inner")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("station_code", "ORIGIN_STATION")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("airport_station_distance", "ORIGIN_STATION_DISTANCE")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("airport_lat", "ORIGIN_LAT")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("airport_lon", "ORIGIN_LON")
    airlines_station_origin_filtered = airlines_station_origin_filtered.drop("airport_code")

    # add closest weather station to airlines dataset
    airlines_station_filtered = airlines_station_origin_filtered.join(airports_closest_station_filtered, airlines_station_origin_filtered.DEST == airports_closest_station_filtered.airport_code, how="inner")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("station_code", "DEST_STATION")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("airport_station_distance", "DEST_STATION_DISTANCE")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("airport_lat", "DEST_LAT")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("airport_lon", "DEST_LON")
    airlines_station_filtered = airlines_station_filtered.drop("airport_code")

    #add flight bearing angle in degrees from true north (consistent with wind direction)
    airlines_station_filtered = airlines_station_filtered.withColumn("FLIGHT_BEARING", udfBearingCalculation_fn("ORIGIN_LAT","ORIGIN_LON","DEST_LAT","DEST_LON"))
    return airlines_station_filtered

def flightDateTimeCalculation_fn(flight_date, flight_time):  
    timestamp_date = str(flight_date)
    timestamp_hour = str(flight_time).zfill(4)[:-2]
    timestamp_minute = str(flight_time).zfill(4)[-2:]
  
    timestamp = timestamp_date + 'T' + timestamp_hour + ':' + timestamp_minute# + ".000+0000"
    try:
      datetime_timestamp = dateutil.parser.isoparse(timestamp)
    except ValueError:
      timestamp = timestamp_date + 'T' + '00' + ':' + timestamp_minute# + ".000+0000"
      datetime_timestamp = dateutil.parser.isoparse(timestamp)
    
    return datetime_timestamp
  
def flightDateTimeCalculationArr_fn(flight_date, flight_time_dep, flight_time_arr):  
    timestamp_dep_date = str(flight_date)
    timestamp_arr_date = str(flight_date)
  
    
    timestamp_dep_hour = str(flight_time_dep).zfill(4)[:-2]
    timestamp_dep_minute = str(flight_time_dep).zfill(4)[-2:]
    timestamp_arr_hour = str(flight_time_arr).zfill(4)[:-2]
    timestamp_arr_minute = str(flight_time_arr).zfill(4)[-2:]
    
    timestamp_dep = timestamp_dep_hour + ':' + timestamp_dep_minute
    timestamp_arr = timestamp_arr_hour + ':' + timestamp_arr_minute
    
    timestamp_dep = timestamp_dep_date + 'T' + timestamp_dep_hour + ':' + timestamp_dep_minute# + ".000+0000"
    try:
      datetime_timestamp_dep = dateutil.parser.isoparse(timestamp_dep)
    except ValueError:
      timestamp_dep = timestamp_dep_date + 'T' + '00' + ':' + timestamp_dep_minute# + ".000+0000"
      datetime_timestamp_dep = dateutil.parser.isoparse(timestamp_dep)
    
    timestamp_arr = timestamp_arr_date + 'T' + timestamp_arr_hour + ':' + timestamp_arr_minute# + ".000+0000"
    try:
      datetime_timestamp_arr = dateutil.parser.isoparse(timestamp_arr)
    except ValueError:
      timestamp_arr = timestamp_arr_date + 'T' + '00' + ':' + timestamp_arr_minute# + ".000+0000"
      datetime_timestamp_arr = dateutil.parser.isoparse(timestamp_arr)
  
    # if flight arrived a later than when started, only works if flight was less than 24 hours long:
    if datetime_timestamp_dep > datetime_timestamp_arr:
      datetime_timestamp_arr = datetime_timestamp_arr + datetime.timedelta(days=1)

    return datetime_timestamp_arr

udfFlightDateTimeCalculation_fn = udf(flightDateTimeCalculation_fn, TimestampType())
udfFlightDateTimeCalculationArr_fn = udf(flightDateTimeCalculationArr_fn, TimestampType())

def airlines_station_datetime_fn(airlines_station_filtered):
    airlines_station_datetime = airlines_station_filtered.withColumn("CRS_DEP_TIMESTAMP", udfFlightDateTimeCalculation_fn("FL_DATE","CRS_DEP_TIME"))
    airlines_station_datetime = airlines_station_datetime.withColumn("CRS_ARR_TIMESTAMP", udfFlightDateTimeCalculationArr_fn("FL_DATE","CRS_DEP_TIME", "CRS_ARR_TIME"))
    return airlines_station_datetime

def airlines_station_datetime_unix_fn(airlines_station_datetime):
    airlines_station_datetime_unix = airlines_station_datetime.withColumn("CRS_DEP_TIMESTAMP_UNIX", f.unix_timestamp("CRS_DEP_TIMESTAMP"))
    airlines_station_datetime_unix = airlines_station_datetime_unix.withColumn("CRS_ARR_TIMESTAMP_UNIX", f.unix_timestamp("CRS_ARR_TIMESTAMP"))
    airlines_station_datetime_unix = airlines_station_datetime_unix.withColumn("DEP_HOUR", f.hour("CRS_DEP_TIMESTAMP"))
    airlines_station_datetime_unix = airlines_station_datetime_unix.withColumn("ARR_HOUR", f.hour("CRS_ARR_TIMESTAMP"))
    
    return airlines_station_datetime_unix
  
def weather_fltr_datetime_fn(weather_fltr_drop):
    weather_fltr_datetime = weather_fltr_drop.withColumn("DATE_TIMESTAMP_UNIX", f.unix_timestamp("TIMESTAMP"))
    weather_fltr_datetime = weather_fltr_datetime.withColumn('DATE', f.col("TIMESTAMP").cast(DateType()))
    weather_fltr_datetime = weather_fltr_datetime.withColumn("HOUR", f.hour("TIMESTAMP"))
    
    return weather_fltr_datetime

def weather_avg_fn(weather_fltr_datetime):
    weather_fltr_datetime.createOrReplaceTempView('weather_time')
    weather_avg_query = "(SELECT STATION, DATE, HOUR, ROUND(AVG(`WND[0]`),0) AS `WND[0]`, ROUND(AVG(`WND[3]`),0) AS `WND[3]`, ROUND(AVG(`VIS[0]`),0) AS `VIS[0]`, ROUND(AVG(`SLP[0]`),0) AS `SLP[0]`, ROUND(AVG(`AA1[0]`),0) AS `AA1[0]` FROM weather_time GROUP BY STATION, DATE, HOUR)"

    weather_avg = spark.sql(weather_avg_query)
    
    weather_avg = weather_avg.withColumn("WND_CLASS[0]", udfBearingClass_fn("WND[0]"))
    weather_avg = weather_avg.drop("WND[0]")
    
    return weather_avg
  
def weather_add_values_fn(weather_avg):
    weather_fltr_datetime_origin = weather_avg.withColumnRenamed("STATION", "ORIGIN_STATION_WEATHER")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("DATE", "ORIGIN_STATION_DATE")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("HOUR", "ORIGIN_STATION_HOUR")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("WND_CLASS[0]", "ORIGIN_STATION_WND[0]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("WND[3]", "ORIGIN_STATION_WND[3]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("VIS[0]", "ORIGIN_STATION_VIS[0]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("SLP[0]", "ORIGIN_STATION_SLP[0]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("AA1[0]", "ORIGIN_STATION_AA1[0]")
    weather_fltr_datetime_dest = weather_avg.withColumnRenamed("STATION", "DEST_STATION_WEATHER")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("DATE", "DEST_STATION_DATE")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("HOUR", "DEST_STATION_HOUR")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("WND_CLASS[0]", "DEST_STATION_WND[0]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("WND[3]", "DEST_STATION_WND[3]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("VIS[0]", "DEST_STATION_VIS[0]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("SLP[0]", "DEST_STATION_SLP[0]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("AA1[0]", "DEST_STATION_AA1[0]")
    return weather_fltr_datetime_origin, weather_fltr_datetime_dest
  
def departure_final_fn(airlines_station_datetime_unix):
    airlines_station_datetime_unix.createOrReplaceTempView("airports_weather")
    weather_fltr_datetime_origin.createOrReplaceTempView("origin_weather")
    origin_join_query = "(SELECT * FROM airports_weather a INNER JOIN origin_weather w ON a.ORIGIN_STATION = w.ORIGIN_STATION_WEATHER AND a.FL_DATE = w.ORIGIN_STATION_DATE AND a.DEP_HOUR = w.ORIGIN_STATION_HOUR)"

    departure_final = spark.sql(origin_join_query)
    return departure_final

def airlines_weather_final_trim_fn(departure_final):
    departure_final.createOrReplaceTempView("airports_weather_dest")
    weather_fltr_datetime_dest.createOrReplaceTempView("dest_weather")
    # chnaged to join on weather @ destination airport @ departure time
    dest_join_query = "(SELECT * FROM airports_weather_dest a INNER JOIN dest_weather w ON a.DEST_STATION = w.DEST_STATION_WEATHER AND a.FL_DATE = w.DEST_STATION_DATE AND a.DEP_HOUR = w.DEST_STATION_HOUR)"

    airlines_weather_final = spark.sql(dest_join_query)
    drop_cols = ['DEST_STATION_DATE', 'DEST_STATION_HOUR', 'ORIGIN_STATION_HOUR', 'ORIGIN_STATION_DATE', 'ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON', 'CRS_DEP_TIMESTAMP_UNIX', 'CRS_ARR_TIMESTAMP_UNIX', 'DEP_HOUR', 'ARR_HOUR', 'ORIGIN_STATION', 'DEST_STATION', 'ORIGIN_STATION_WEATHER', 'DEST_STATION_WEATHER']
    airlines_weather_final_trim = airlines_weather_final.drop(*drop_cols)
    return airlines_weather_final_trim
  
def airlines_weather_to_parquet_fn(airlines_weather_final_trim):
    dbutils.fs.rm("dbfs:/tmp/parquet/airlines_weather_final_4_7.parquet")
    airlines_weather_final_trim.write.parquet("dbfs:/tmp/parquet/airlines_weather_final_4_7.parquet")
    return None

In [25]:
# Merging airlines and weather data within United States
airlines_df =  preprocessAirlines(airlines)
weather_parquet_us = US_fn(weather_parquet)
weather_fltr_drop = reduce_split_cols_fn(weather_parquet_us)
weather_fltr_drop_distinct = distinct_station_fn(weather_fltr_drop)
airport_codes_us = US_fn(airport_codes)
airports_stations_distance_all = haversine_join_station_aircode_fn(airport_codes_us, weather_fltr_drop_distinct)
airports_closest_station_filtered = airports_closest_stations_fn(airports_stations_distance_all)
airports_closest_station_filtered = join_closest_weather_airlines_fn(airlines_df, airports_closest_station_filtered)
airlines_station_datetime = airlines_station_datetime_fn(airports_closest_station_filtered)
airlines_station_datetime_unix = airlines_station_datetime_unix_fn(airlines_station_datetime)  
weather_fltr_datetime = weather_fltr_datetime_fn(weather_fltr_drop)
weather_avg = weather_avg_fn(weather_fltr_datetime)
weather_fltr_datetime_origin, weather_fltr_datetime_dest = weather_add_values_fn(weather_avg)
departure_final = departure_final_fn(airlines_station_datetime_unix)
airlines_weather_final_trim = airlines_weather_final_trim_fn(departure_final)

In [26]:
airlines_preprocessed_nullCounts_df = nullDataFrame(airlines_df)
airlines_preprocessed_nullCounts_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,MONTH,0,0.0
1,DAY_OF_WEEK,0,0.0
2,FL_DATE,0,0.0
3,OP_UNIQUE_CARRIER,0,0.0
4,ORIGIN,0,0.0
5,DEST,0,0.0
6,CRS_DEP_TIME,0,0.0
7,DEP_TIME,0,0.0
8,DEP_DELAY,0,0.0
9,DEP_TIME_BLK,0,0.0


In [27]:
#airlines_weather_final_trim.write.parquet("/FileStore/tables/airlines_weather_final_trim.parquet")
airlines_weather_final_trim = spark.read.parquet("/FileStore/tables/airlines_weather_final_trim.parquet")

In [28]:
airlines_weather_final_trim.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: date (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- ORIGIN: string (nullable = true)
-- DEST: string (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- CARRIER_DELAY: integer (nullable = true)
-- WEATHER_DELAY: integer (nullable = true)
-- NAS_DELAY: integer (nullable = true)
-- SECURITY_DELAY: integer (nullable = true)
-- LATE_AIRCRAFT_DELAY: integer (nullable = true)
-- IS_WEEKEND: integer (nullable = true)
-- DEP_RUSH_HOUR: integer (nullable = true)
-- ARR_RUSH_HOUR: integer (nullable = true)
-- ORIGIN_CARRIER: string (nullable = true)
-- DEST_CARRIER: string (nullable = true)
-- ORIGIN_STATION_DISTANCE: double (nullable = true)
-- DEST_STATION_DISTANCE: double (nullable = true)
-- FLIGHT_BEARING: string (nullable = true)
-- CRS_DEP_TIMESTAMP: timestamp (nullable = true)
-- CRS_ARR_TIMESTAMP: timestamp (nullable = true)
-- ORIGIN_STATION_WND[3]: double (nullable = true)
-- ORIGIN_STATION_VIS[0]: double (nullable = true)
-- ORIGIN_STATION_SLP[0]: double (nullable = true)
-- ORIGIN_STATION_AA1[0]: double (nullable = true)
-- ORIGIN_STATION_WND[0]: string (nullable = true)
-- DEST_STATION_WND[3]: double (nullable = true)
-- DEST_STATION_VIS[0]: double (nullable = true)
-- DEST_STATION_SLP[0]: double (nullable = true)
-- DEST_STATION_AA1[0]: double (nullable = true)
-- DEST_STATION_WND[0]: string (nullable = true)

#### Data Split:

In [30]:
airlines_train, airlines_val, airlines_test = airlines_weather_final_trim.randomSplit([0.8,0.1,0.1], seed = 2020)

In [31]:
train_cnt = airlines_train.count()
val_cnt = airlines_val.count()
test_cnt = airlines_test.count()
total_cnt = train_cnt + val_cnt + test_cnt
print('airlines_train records: {}\n airlines_val records: {}\n  airlines_test records: {}\n total records: {}'.format(train_cnt, val_cnt, test_cnt, total_cnt) )

airlines_train records: 109399
 airlines_val records: 13859
 airlines_test records: 13569
 total records: 136827

## Feature Engineering & Feature Selection:

In [33]:
def featureSelection(df):
  cols_to_keep = ['MONTH', 'DAY_OF_WEEK', 'DEP_DELAY', 'DEP_TIME_BLK', 'ARR_DELAY', 'ARR_TIME_BLK', 'CRS_ELAPSED_TIME', 'DISTANCE',  'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'IS_WEEKEND', 'DEP_RUSH_HOUR', 'ARR_RUSH_HOUR','FLIGHT_BEARING', 'ORIGIN_CARRIER', 'DEST_CARRIER', 'ORIGIN_STATION_WND_DIR', 'ORIGIN_STATION_VIS', 'ORIGIN_STATION_SLP','ORIGIN_STATION_AA1',    'ORIGIN_STATION_WND', 'DEST_STATION_WND_DIR', 'DEST_STATION_VIS', 'DEST_STATION_SLP', 'DEST_STATION_AA1',  'DEST_STATION_WND']
  cols_to_remove = [x for x in df.columns if x not in cols_to_keep]
  df = df.withColumnRenamed("ORIGIN_STATION_WND[0]", "ORIGIN_STATION_WND_DIR")
  df = df.withColumnRenamed("ORIGIN_STATION_VIS[0]", "ORIGIN_STATION_VIS")
  df = df.withColumnRenamed("ORIGIN_STATION_SLP[0]", "ORIGIN_STATION_SLP")
  df = df.withColumnRenamed("ORIGIN_STATION_AA1[0]", "ORIGIN_STATION_AA1")
  df = df.withColumnRenamed("ORIGIN_STATION_WND[3]", "ORIGIN_STATION_WND")
  
  df = df.withColumnRenamed("DEST_STATION_WND[0]", "DEST_STATION_WND_DIR")
  df = df.withColumnRenamed("DEST_STATION_VIS[0]", "DEST_STATION_VIS")
  df = df.withColumnRenamed("DEST_STATION_SLP[0]", "DEST_STATION_SLP")
  df = df.withColumnRenamed("DEST_STATION_AA1[0]", "DEST_STATION_AA1")
  df = df.withColumnRenamed("DEST_STATION_WND[3]", "DEST_STATION_WND")  
  
  featureSelection_df = df.drop(*cols_to_remove)
  return featureSelection_df

In [34]:
airlines_train_df =  featureSelection(airlines_train)

In [35]:
numeric_features = [x[0] for x in airlines_train_df.dtypes if x[1] == 'int' or x[1] == 'double']
numeric_features.remove('ARR_DELAY')
cat_features = ['MONTH', 'DAY_OF_WEEK'] # These are catogorical fetures in 'int' type
numeric_features = [x for x in numeric_features if x not in cat_features]
numeric_features

Out[29]: ['DEP_DELAY',
 'CRS_ELAPSED_TIME',
 'DISTANCE',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'IS_WEEKEND',
 'DEP_RUSH_HOUR',
 'ARR_RUSH_HOUR',
 'ORIGIN_STATION_WND',
 'ORIGIN_STATION_VIS',
 'ORIGIN_STATION_SLP',
 'ORIGIN_STATION_AA1',
 'DEST_STATION_WND',
 'DEST_STATION_VIS',
 'DEST_STATION_SLP',
 'DEST_STATION_AA1']

In [36]:
categorical_features = [x[0] for x in airlines_train_df.dtypes if x[1] == 'string']
categorical_features = categorical_features + cat_features
categorical_features

Out[30]: ['DEP_TIME_BLK',
 'ARR_TIME_BLK',
 'ORIGIN_CARRIER',
 'DEST_CARRIER',
 'FLIGHT_BEARING',
 'ORIGIN_STATION_WND_DIR',
 'DEST_STATION_WND_DIR',
 'MONTH',
 'DAY_OF_WEEK']

In [37]:
stages = []
for categoricalCol in categorical_features:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index', handleInvalid="keep")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
assemblerInputs = [c + "classVec" for c in categorical_features] + numeric_features
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features", handleInvalid="keep")
stages += [assembler]

In [38]:
pipeline = Pipeline().setStages(stages).fit(airlines_train_df)
vector_airlines_train_df = pipeline.transform(airlines_train_df)
vector_airlines_train_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- CARRIER_DELAY: integer (nullable = true)
-- WEATHER_DELAY: integer (nullable = true)
-- NAS_DELAY: integer (nullable = true)
-- SECURITY_DELAY: integer (nullable = true)
-- LATE_AIRCRAFT_DELAY: integer (nullable = true)
-- IS_WEEKEND: integer (nullable = true)
-- DEP_RUSH_HOUR: integer (nullable = true)
-- ARR_RUSH_HOUR: integer (nullable = true)
-- ORIGIN_CARRIER: string (nullable = true)
-- DEST_CARRIER: string (nullable = true)
-- FLIGHT_BEARING: string (nullable = true)
-- ORIGIN_STATION_WND: double (nullable = true)
-- ORIGIN_STATION_VIS: double (nullable = true)
-- ORIGIN_STATION_SLP: double (nullable = true)
-- ORIGIN_STATION_AA1: double (nullable = true)
-- ORIGIN_STATION_WND_DIR: string (nullable = true)
-- DEST_STATION_WND: double (nullable = true)
-- DEST_STATION_VIS: double (nullable = true)
-- DEST_STATION_SLP: double (nullable = true)
-- DEST_STATION_AA1: double (nullable = true)
-- DEST_STATION_WND_DIR: string (nullable = true)
-- DEP_TIME_BLKIndex: double (nullable = false)
-- DEP_TIME_BLKclassVec: vector (nullable = true)
-- ARR_TIME_BLKIndex: double (nullable = false)
-- ARR_TIME_BLKclassVec: vector (nullable = true)
-- ORIGIN_CARRIERIndex: double (nullable = false)
-- ORIGIN_CARRIERclassVec: vector (nullable = true)
-- DEST_CARRIERIndex: double (nullable = false)
-- DEST_CARRIERclassVec: vector (nullable = true)
-- FLIGHT_BEARINGIndex: double (nullable = false)
-- FLIGHT_BEARINGclassVec: vector (nullable = true)
-- ORIGIN_STATION_WND_DIRIndex: double (nullable = false)
-- ORIGIN_STATION_WND_DIRclassVec: vector (nullable = true)
-- DEST_STATION_WND_DIRIndex: double (nullable = false)
-- DEST_STATION_WND_DIRclassVec: vector (nullable = true)
-- MONTHIndex: double (nullable = false)
-- MONTHclassVec: vector (nullable = true)
-- DAY_OF_WEEKIndex: double (nullable = false)
-- DAY_OF_WEEKclassVec: vector (nullable = true)
-- features: vector (nullable = true)

In [39]:
train_df = vector_airlines_train_df.select(col("ARR_DELAY").alias("label"), col("features"))
train_df.show(2)

+-----+--------------------+
label| features|
+-----+--------------------+
 -4.0|(3818,[14,26,43,2...|
 6.0|(3818,[9,30,42,21...|
+-----+--------------------+
only showing top 2 rows

In [40]:
airlines_val_df =  featureSelection(airlines_val)
vector_airlines_val_df = pipeline.transform(airlines_val_df)
val_df = vector_airlines_val_df.select(col("ARR_DELAY").alias("label"), col("features"))

<h2 align="center">Algorithm Exploration:</h2>
To predict `ARRIVAL_DELAY` from the dataset, we are going to consider below supervised machine learning algorithms using cross validation.

1. Linear Regression
2. Decision Tree Regressor
3. Random Forest Regressor
4. Gradient Boosted Tree Regressor

### Linear Regression:

In [43]:
lr = LinearRegression(featuresCol = 'features', labelCol='label')
paramGrid_lr = ParamGridBuilder() \
   .addGrid(lr.regParam, [0.1, 0.01, 0.001]) \
   .addGrid(lr.maxIter, [10, 20]) \
   .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
   .build() 

crossval_lr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_lr = crossval_lr.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [44]:
regression_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="r2")
regression_evaluator_rmse = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="rmse")
regression_evaluator_mae = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="mae")

regression_metrics_list = []

In [45]:
# Save and Load CrossValidatorModel
cvModel_lr.write().overwrite().save('/FileStore/tables/cvModel_lr')

In [46]:
saved_cvModel_lr = CrossValidatorModel.load('/FileStore/tables/cvModel_lr')

In [47]:
# train_df evaluation metrics
lr_predictions_train = saved_cvModel_lr.transform(train_df)
lr_train_r2 = regression_evaluator_r2.evaluate(lr_predictions_train)
lr_train_rmse = regression_evaluator_rmse.evaluate(lr_predictions_train)
lr_train_mae = regression_evaluator_mae.evaluate(lr_predictions_train)
regression_metrics_list.append(["LinearRegression_TrainData_CV", lr_train_r2, lr_train_rmse, lr_train_mae ])

#  val_df evaluation metrics
lr_predictions_val = saved_cvModel_lr.transform(val_df)
lr_val_r2 = regression_evaluator_r2.evaluate(lr_predictions_val)
lr_val_rmse = regression_evaluator_rmse.evaluate(lr_predictions_val)
lr_val_mae = regression_evaluator_mae.evaluate(lr_predictions_val)
regression_metrics_list.append(["LinearRegression_ValData_CV", lr_val_r2, lr_val_rmse, lr_val_mae ])

In [48]:
bestLRModel = cvModel_lr.bestModel
bestParams = bestLRModel.extractParamMap()
bestParams

Out[40]: {Param(parent='LinearRegression_a549544751b2', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LinearRegression_a549544751b2', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.0,
 Param(parent='LinearRegression_a549544751b2', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'): 1.35,
 Param(parent='LinearRegression_a549544751b2', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LinearRegression_a549544751b2', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LinearRegression_a549544751b2', name='labelCol', doc='label column name'): 'label',
 Param(parent='LinearRegression_a549544751b2', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)'): 'squaredError',
 Param(parent='LinearRegression_a549544751b2', name='maxIter', doc='maximum number of iterations (>= 0)'): 10,
 Param(parent='LinearRegression_a549544751b2', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='LinearRegression_a549544751b2', name='regParam', doc='regularization parameter (>= 0)'): 0.01,
 Param(parent='LinearRegression_a549544751b2', name='solver', doc='The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (Default auto)'): 'auto',
 Param(parent='LinearRegression_a549544751b2', name='standardization', doc='whether to standardize the training features before fitting the model'): True,
 Param(parent='LinearRegression_a549544751b2', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0)'): 1e-06}

### Decision Tree Regressor

In [50]:
dt = DecisionTreeRegressor(featuresCol="features", labelCol='label') 

paramGrid_dt = ParamGridBuilder()\
    .addGrid(dt.maxBins, [16, 32]) \
    .addGrid(dt.maxDepth, [5, 10]) \
    .addGrid(dt.minInstancesPerNode, [1, 5]) \
    .build()  

crossval_dt = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid_dt,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_dt = crossval_dt.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [51]:
# Save and Load CrossValidatorModel
cvModel_dt.write().overwrite().save('/FileStore/tables/cvModel_dt')

In [52]:
saved_cvModel_dt = CrossValidatorModel.load('/FileStore/tables/cvModel_dt')

In [53]:
# train_df evaluation metrics
dt_predictions_train = saved_cvModel_dt.transform(train_df)
dt_train_r2 = regression_evaluator_r2.evaluate(dt_predictions_train)
dt_train_rmse = regression_evaluator_rmse.evaluate(dt_predictions_train)
dt_train_mae = regression_evaluator_mae.evaluate(dt_predictions_train)
regression_metrics_list.append(["DecisionTreeRegressor_TrainData_CV", dt_train_r2, dt_train_rmse, dt_train_mae ])

# val_df evaluation metrics
dt_predictions_val = saved_cvModel_dt.transform(val_df)
dt_val_r2 = regression_evaluator_r2.evaluate(dt_predictions_val)
dt_val_rmse = regression_evaluator_rmse.evaluate(dt_predictions_val)
dt_val_mae = regression_evaluator_mae.evaluate(dt_predictions_val)
regression_metrics_list.append(["DecisionTreeRegressor_ValData_CV", dt_val_r2, dt_val_rmse, dt_val_mae ])

In [54]:
bestDTModel = cvModel_dt.bestModel
bestParams_dt = bestDTModel.extractParamMap()
bestParams_dt

Out[45]: {Param(parent='DecisionTreeRegressor_119a9c851016', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='DecisionTreeRegressor_119a9c851016', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='DecisionTreeRegressor_119a9c851016', name='featuresCol', doc='features column name'): 'features',
 Param(parent='DecisionTreeRegressor_119a9c851016', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='DecisionTreeRegressor_119a9c851016', name='labelCol', doc='label column name'): 'label',
 Param(parent='DecisionTreeRegressor_119a9c851016', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='DecisionTreeRegressor_119a9c851016', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='DecisionTreeRegressor_119a9c851016', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='DecisionTreeRegressor_119a9c851016', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='DecisionTreeRegressor_119a9c851016', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 5,
 Param(parent='DecisionTreeRegressor_119a9c851016', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='DecisionTreeRegressor_119a9c851016', name='seed', doc='random seed'): -1407754390808368278}

### Random Forest Regressor

In [56]:
rf = RandomForestRegressor(featuresCol="features", labelCol='label')

paramGrid_rf = ParamGridBuilder()\
    .addGrid(rf.maxBins, [16, 32]) \
    .addGrid(rf.numTrees, [20, 40]) \
    .addGrid(rf.minInstancesPerNode, [1, 5]) \
    .build()  

crossval_rf = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid_rf,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_rf = crossval_rf.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [57]:
  # Save and Load CrossValidatorModel
cvModel_rf.write().overwrite().save('/FileStore/tables/cvModel_rf')


In [58]:
saved_cvModel_rf = CrossValidatorModel.load('/FileStore/tables/cvModel_rf')

In [59]:
rf_predictions_train = saved_cvModel_rf.transform(train_df)
rf_train_r2 = regression_evaluator_r2.evaluate(rf_predictions_train)
rf_train_rmse = regression_evaluator_rmse.evaluate(rf_predictions_train)
rf_train_mae = regression_evaluator_mae.evaluate(rf_predictions_train)
regression_metrics_list.append(["RandomForestRegressor_TrainData_CV", rf_train_r2, rf_train_rmse, rf_train_mae ])

# val_df evaluation metrics
rf_predictions_val = saved_cvModel_rf.transform(val_df)
rf_val_r2 = regression_evaluator_r2.evaluate(rf_predictions_val)
rf_val_rmse = regression_evaluator_rmse.evaluate(rf_predictions_val)
rf_val_mae = regression_evaluator_mae.evaluate(rf_predictions_val)
regression_metrics_list.append(["RandomForestRegressor_ValData_CV", rf_val_r2, rf_val_rmse, rf_val_mae ])

In [60]:
bestRFModel = cvModel_rf.bestModel
bestParams_rf = bestRFModel.extractParamMap()
bestParams_rf

Out[50]: {Param(parent='RandomForestRegressor_35a81bc4637b', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto',
 Param(parent='RandomForestRegressor_35a81bc4637b', name='featuresCol', doc='features column name'): 'features',
 Param(parent='RandomForestRegressor_35a81bc4637b', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='RandomForestRegressor_35a81bc4637b', name='labelCol', doc='label column name'): 'label',
 Param(parent='RandomForestRegressor_35a81bc4637b', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 5,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='numTrees', doc='Number of trees to train (at least 1)'): 20,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='RandomForestRegressor_35a81bc4637b', name='seed', doc='random seed'): 2502083311556356884,
 Param(parent='RandomForestRegressor_35a81bc4637b', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].'): 1.0}

### Gradient-Boosted Trees

In [62]:
 gbt = GBTRegressor(featuresCol="features", labelCol='label')

paramGrid_gbt = ParamGridBuilder()\
    .addGrid(gbt.maxBins, [10, 32]) \
    .addGrid(gbt.minInstancesPerNode, [1, 5]) \
    .build()  

crossval_gbt = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid_gbt,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_gbt = crossval_gbt.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [63]:
# Save and Load CrossValidatorModel
cvModel_gbt.write().overwrite().save('/FileStore/tables/cvModel_gbt')


In [64]:
saved_cvModel_gbt = CrossValidatorModel.load('/FileStore/tables/cvModel_gbt')

In [65]:
gbt_predictions_train = saved_cvModel_gbt.transform(train_df)
gbt_train_r2 = regression_evaluator_r2.evaluate(gbt_predictions_train)
gbt_train_rmse = regression_evaluator_rmse.evaluate(gbt_predictions_train)
gbt_train_mae = regression_evaluator_mae.evaluate(gbt_predictions_train)
regression_metrics_list.append(["GradientBoostedTreeRegressor_TrainData_CV", gbt_train_r2, gbt_train_rmse, gbt_train_mae ])

# val_df evaluation metrics
gbt_predictions_val = saved_cvModel_gbt.transform(val_df)
gbt_val_r2 = regression_evaluator_r2.evaluate(gbt_predictions_val)
gbt_val_rmse = regression_evaluator_rmse.evaluate(gbt_predictions_val)
gbt_val_mae = regression_evaluator_mae.evaluate(gbt_predictions_val)
regression_metrics_list.append(["GradientBoostedTreeRegressor_ValData_CV", gbt_val_r2, gbt_val_rmse, gbt_val_mae ])

In [66]:
bestGBTModel = cvModel_gbt.bestModel
bestParams_gbt = bestGBTModel.extractParamMap()
bestParams_gbt

Out[55]: {Param(parent='GBTRegressor_966e93709412', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='GBTRegressor_966e93709412', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='GBTRegressor_966e93709412', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'all',
 Param(parent='GBTRegressor_966e93709412', name='featuresCol', doc='features column name'): 'features',
 Param(parent='GBTRegressor_966e93709412', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='GBTRegressor_966e93709412', name='labelCol', doc='label column name'): 'label',
 Param(parent='GBTRegressor_966e93709412', name='lossType', doc='Loss function which GBT tries to minimize (case-insensitive). Supported options: squared, absolute'): 'squared',
 Param(parent='GBTRegressor_966e93709412', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='GBTRegressor_966e93709412', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='GBTRegressor_966e93709412', name='maxIter', doc='maximum number of iterations (>= 0)'): 20,
 Param(parent='GBTRegressor_966e93709412', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='GBTRegressor_966e93709412', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='GBTRegressor_966e93709412', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 5,
 Param(parent='GBTRegressor_966e93709412', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='GBTRegressor_966e93709412', name='seed', doc='random seed'): -6682481135904123338,
 Param(parent='GBTRegressor_966e93709412', name='stepSize', doc='Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator.'): 0.1,
 Param(parent='GBTRegressor_966e93709412', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].'): 1.0}

## Results:

In [68]:
regression_metrics_df = pd.DataFrame(regression_metrics_list, columns = ['Model_Data' , 'R^2', 'RMSE', 'MAE']) 
display(regression_metrics_df)

Model_Data,R^2,RMSE,MAE
LinearRegression_TrainData_CV,0.9556805965871404,13.422721977071546,9.542329680250475
LinearRegression_ValData_CV,0.9578701682031706,13.397124217962183,9.657731791586379
DecisionTreeRegressor_TrainData_CV,0.7762539878356634,29.88296529022086,11.616142687603576
DecisionTreeRegressor_ValData_CV,0.7428123335791165,33.48911533037165,11.402500854607558
RandomForestRegressor_TrainData_CV,0.70284374595401,34.934630735911,13.792047792327292
RandomForestRegressor_ValData_CV,0.6815277669348756,36.31649598510621,13.553556662155774
GradientBoostedTreeRegressor_TrainData_CV,0.7553940164461354,30.382756887520873,11.892415663469043
GradientBoostedTreeRegressor_ValData_CV,0.7765612911647015,33.29219070666156,12.084116229875963


From above results, we can see that Linear Regression performed well on the training and validation data. We will consider Linear Regression model for our inference on test data.

<h2 align="center">Algorithm Implementation:</h2>

#### Inference:
We will now predict the `ARRIVAL_DELAY` for test data using Linear Regression model that we trained using cross validation and hyperparameter tuning.

In [72]:
airlines_test_df =  featureSelection(airlines_test)
vector_airlines_test_df = pipeline.transform(airlines_test_df)
test_df = vector_airlines_test_df.select(col("ARR_DELAY").alias("label"), col("features"))

In [73]:
# # test_df evaluation metrics
lr_predictions_test = saved_cvModel_lr.transform(test_df)
lr_test_r2 = regression_evaluator_r2.evaluate(lr_predictions_test)
lr_test_rmse = regression_evaluator_rmse.evaluate(lr_predictions_test)
lr_test_mae = regression_evaluator_mae.evaluate(lr_predictions_test)
regression_metrics_list.append(["LinearRegression_TestData_CV", lr_test_r2, lr_test_rmse, lr_test_mae ])

In [74]:
regression_metrics_test_df = pd.DataFrame(regression_metrics_list, columns = ['Model_Data' , 'R^2', 'RMSE', 'MAE']) 
display(regression_metrics_test_df)

Model_Data,R^2,RMSE,MAE
LinearRegression_TrainData_CV,0.9556805965871404,13.422721977071546,9.542329680250475
LinearRegression_ValData_CV,0.9578701682031706,13.397124217962183,9.657731791586379
DecisionTreeRegressor_TrainData_CV,0.7762539878356634,29.88296529022086,11.616142687603576
DecisionTreeRegressor_ValData_CV,0.7428123335791165,33.48911533037165,11.402500854607558
RandomForestRegressor_TrainData_CV,0.70284374595401,34.934630735911,13.792047792327292
RandomForestRegressor_ValData_CV,0.6815277669348756,36.31649598510621,13.553556662155774
GradientBoostedTreeRegressor_TrainData_CV,0.7553940164461354,30.382756887520873,11.892415663469043
GradientBoostedTreeRegressor_ValData_CV,0.7765612911647015,33.29219070666156,12.084116229875963
LinearRegression_TestData_CV,0.9529835434342335,13.605171894321128,9.520017398426202


<h2 align="center">Conclusion:</h2>

<h2 align="center">Application of Course Concepts:</h2>

In [77]:
r = ChiSquareTest.test(train_df, "features", "label").head()
print("pValues: " + str(r.pValues))


pValues: [0.0,0.9827346511349574,0.009544473864153358,0.35992115064902197,0.0,0.9837804784351413,0.03103345394770185,0.9998638678931697,0.0,0.6179170946982969,4.6629367034256575e-15,0.0,0.0,0.9741533908025837,0.0,0.18462806958171396,5.975889316722771e-07,0.0,1.9984014443252818e-15,0.9887376604010094,1.3223251382754597e-09,0.7916437182396486,8.208152778788058e-05,0.054316212235630656,0.9993558452333587,0.0,0.0,0.5490896616465488,0.9989596784055008,0.0,0.36082689261385903,1.5911392003964409e-07,6.894484982922222e-12,0.0,4.033662293068119e-12,0.0,0.0269447460380009,0.0,1.5866427838240327e-08,0.0,1.0,1.643777958193482e-08,9.024157638926056e-07,1.0,0.0005490775520533564,0.9999681177920492,0.0,0.0,1.0,1.0,2.201979660221287e-05,0.9999999999999638,4.313216450668733e-13,1.0,0.0,1.0,0.9999999723683993,1.0,0.008263856791166702,1.0,4.9960036108132044e-15,0.0,1.0,1.0,0.0005515331454098993,1.0,0.0,0.03725528238796694,1.0,0.9999999999999882,1.0,0.9999999999979362,0.04664218340191628,0.9999999999999998,0.9999999999996156,0.0023078034480493725,0.0,0.9999997893195042,1.0,0.7368168068523818,0.9999984446253608,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.31078210862254174,0.9999999981581043,1.0,0.0,1.0,1.0,1.0,0.0,0.999754923694343,0.22211137487166877,1.0,1.0,1.0,0.0,1.0,0.9999999997958386,0.0037462872502941336,0.22590729283953048,0.9997408502571664,0.9999999992231711,1.0,1.0,0.0,1.0,0.9999999999996328,1.0,0.0,1.0,1.0,1.0,0.9999999992866837,0.0,0.0,0.0,0.9999999999998971,0.9999999100615855,0.9999999999999479,1.0,0.004110428129658561,1.0,4.884981308350689e-14,1.0,2.713606006565783e-10,0.0037380380706022454,0.9999999999709854,0.9999999987887707,0.0,0.9999999999999962,0.0,0.0,0.9999999833798277,1.0,0.9999819922768735,1.0,1.0,1.0,1.0,0.9991157560859568,1.0,0.0,0.11203002209827528,1.0,1.0,0.9999983470401926,4.92949369843565e-05,0.0,1.0,1.0,1.8680256008707374e-08,1.3322676295501878e-15,1.0,1.0,1.0,0.6286364520817997,0.0,1.0,0.8742388715692855,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.3964129852439555e-10,0.0,0.0,4.7455229190696e-08,1.0,0.0,0.0,0.0,0.004910212133327652,0.9948137773917142,0.9900433406945711,0.0,1.0,0.0,1.0,0.9999995564648709,1.0,0.7959358685171827,0.0,1.0,1.0,0.9999999999999933,0.99373127626487,0.9999181461313589,0.9946775101737149,0.0,0.00010870526246953283,0.0,1.0,0.30483262627241514,1.0,1.0,1.0,1.0,1.0,0.0,0.001337286563628859,1.0,1.0,1.0,1.0,0.9978329694155667,0.0,0.9999999922975955,1.0,1.0,1.0,0.0,1.0,0.0,5.865047336683915e-10,1.0,1.0,0.9744535225614692,0.986891218396573,1.0,0.0,1.0,2.006483867944553e-10,0.0,0.0,1.0,0.0,1.0,0.02590573661432849,1.0,1.0,0.9999992454328138,0.9999999999992675,1.0,1.0,0.6885574893850059,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.2034453039943003e-08,0.9999999978955433,1.0,0.9999998997666605,0.9951071628781855,1.0,0.0,0.9910769693526105,0.6667506142042576,0.9999999997899929,0.7943699112149646,1.3913314944602462e-12,1.0,1.0,0.23124821022383268,0.0,3.6637359812630166e-15,0.9999999999972812,1.0,1.0,0.9245916678512159,1.0,1.0,0.0,0.0,0.9999990563916215,0.8211433615781499,1.0,1.0,1.0,0.0,1.0,0.999999999999929,0.0,0.0,1.0,1.0,1.0,0.0,0.999999999999963,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0056692171655409185,1.0,0.999999999996054,1.0,0.9999884546762836,0.0,1.0,0.6559712136686855,0.0,0.0,1.0,0.9999813334206085,0.9999999995894714,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.9999999999999922,0.0,0.0,0.0,1.0,0.9999994419967014,1.0,1.0,0.8394017016651371,1.0,0.31409932358051307,1.0,1.0,1.0,1.0,1.0,0.0,0.9999996854442182,1.0,0.9527660675900012,0.9999991694887027,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.9999999760843377,0.6994675129624284,1.0,0.012371643327224069,1.0,0.9999999923327293,1.0,1.0,1.0,0.4822984910911645,0.0,1.0,1.0,0.9999949567540113,0.0006024372290792668,0.0,0.26139551214481904,0.0,0.0,1.0,1.0,0.9999998426149332,1.0,0.0,1.0,0.0,1.0,0.9999999999979265,0.0,1.0,0.9887533677633829,1.0,0.9999057331023906,1.0,0.0,0.9999999999999998,0.9999999999999962,0.0,1.1064379165981464e-05,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.5067053918879512,1.0,7.47

In [78]:
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
 

degreesOfFreedom: [731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 731, 7

In [79]:

print("statistics: " + str(r.statistics))

statistics: [1157.2390654660637,652.5029572832798,823.6019469074892,744.1261631608729,1598.1180173832875,651.6125099182992,803.9764095339835,599.9048309016624,1216.0251494108215,718.9280669455234,1067.7920050250498,1438.5043476085525,1345.1975621660836,658.4796928004116,1205.541054996784,765.1856354341129,931.9699835974849,1526.6577781146627,1073.107867584773,646.5824622825952,982.009799786307,699.7350891360344,883.9726987167597,793.373438926595,614.1397750577208,1356.123030030225,1246.9837491988249,725.6292187759861,618.9181157907615,1403.3662388065325,744.0325194753423,943.5021654526583,1019.9969628471846,1224.3910005589944,1023.6781155252868,1324.6341557161647,806.500667525764,1463.0210973200958,962.5987808032754,1288.765526054932,413.3426243600959,962.3142329751114,928.2804410864751,446.71276294482993,862.2744163102918,587.9968403339735,1367.1662583945667,1451.5593752234734,454.0435214135499,437.1645007290683,897.7701044843168,480.39492232705754,1038.7052225809582,323.07649796564266,1233.7952643213828,289.7710568592215,541.8846158542129,374.8040989245174,825.8081757231055,281.9545522521802,1067.3400292312745,1332.2641245776865,453.61299909890863,367.4426322054914,862.2204789215141,397.8941465035499,1935.1320958370757,800.626576816192,403.2976510771237,476.19926293767,216.23847568091549,496.4162970552017,796.3630720341082,461.98662742571037,489.5682324905768,844.0012954150176,1216.7590724844727,553.6695823252687,290.9887924511269,706.3882061024996,566.3116020014883,416.5843678365024,336.1588334279118,241.55790656745813,1964.3517845543515,349.95500133348423,296.437500449223,371.6584580379046,749.3608543702613,527.5107043134017,342.33489851984336,1713.449545080268,263.7593022783862,221.76950163293955,354.7535023569234,1382.2554374893286,605.085239825868,759.9631337568057,216.31523324135844,254.26849121849148,338.5081572420985,1590.333740887469,240.13094878283985,516.7587222506576,837.3484760612076,759.4653278983848,605.5893548721391,523.2011636447287,396.3228650541843,222.04124889108857,1203.9738532176586,227.32984776158858,489.38522481847673,382.61083674760033,1145.7554952338107,244.1352556974827,204.59371680545817,225.50777656346955,522.7818668101804,1166.1377457603257,1145.136301975439,1458.8992848238777,484.3923990234155,548.6135911663698,481.77653340844125,193.45089448478384,836.0412722128495,258.8750157060045,1052.886194256489,354.3124194344502,993.8511066032827,837.3794037548464,507.802653889691,525.4032531900745,1123.6183766881366,472.0204112361631,1106.7636811058953,1768.2923007893767,539.0788851559156,456.7615526746659,583.6074593582788,225.56485771095927,388.2947543414255,274.0852490143715,433.87414842194244,617.2743016308307,222.36305006976556,1443.416406003196,777.7868378091375,301.48467963799357,376.70501936458635,566.7151995787018,889.4224055779565,1353.6208047868415,283.2250803301244,324.7478832519399,961.2835485042389,1075.3785406686943,444.16185260415153,319.7178213011982,227.6997078993942,717.861063242768,1163.1671977831359,261.1837059394579,687.3850328099833,339.65133083752477,1590.5328734092875,255.5815451994897,352.4558218548104,235.61077174516865,1330.2878536443154,319.7272615468097,1691.0204033777884,301.9870941639438,221.35145867113278,158.7325605709947,337.5736690388069,1822.4006613319918,438.4742957680105,1229.587522136195,214.77829255442128,998.7097721265194,1888.2662798996694,2770.1225722130926,953.668470585438,184.3540775427147,1255.0824428092826,1358.4761992556364,1614.1767882205993,833.5035853764354,636.7100491291366,644.9434594841018,1504.5721805526018,130.5144536054116,1560.3391221371633,369.79106672305466,558.2440596236958,126.06898646258114,699.1761451180878,1838.7380253214806,172.09986084228208,229.32698022092714,474.101510197136,639.0314024885415,595.5980615413959,637.0244138195766,2128.2833171402867,880.9094583062496,1536.985977492819,359.4851733365988,750.0186996024102,454.262135544692,140.71534720164618,161.82865105079821,334.61963202070535,331.7125251583094,1885.7574836318506,851.1780